<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/5_GraphDBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.9/311.9 kB 5.4 MB/s eta 0:00:00


In [3]:
!pip install yfiles_jupyter_graphs

In [4]:
from neo4j import GraphDatabase

[https://sandbox.neo4j.com/](https://sandbox.neo4j.com/)

In [5]:
driver = GraphDatabase.driver("bolt://INSERTBOLTURL:PORT", auth=("neo4j", "<INSERTPASSWORD>"))
session = driver.session(database="neo4j")
driver.execute_query("MATCH (n) DETACH DELETE n  ") # purging eventual graphs already loaded

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x79c9ee0eb2d0>, keys=[])

In [6]:
driver.execute_query(
    "CREATE (john:Person {name: \"John\"})   \
    CREATE (joe:Person {name: \"Joe\"})      \
    CREATE (steve:Person {name: \"Steve\"})  \
    CREATE (sara:Person {name: \"Sara\"})    \
    CREATE (maria:Person {name: \"Maria\"})  \
    CREATE (john)-[:FRIEND]->(joe)-[:FRIEND]->(steve)    \
    CREATE (john)-[:FRIEND]->(sara)-[:FRIEND]->(maria)   \
")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x79c9ee106a10>, keys=[])

In [7]:
results = driver.execute_query("MATCH (n) return n")

In [8]:
#printing all the properties "name"
for record in results[0]:
    elem = record["n"]
    print("{} node with name {}".format(list(elem.labels)[0], elem["name"]))

Person node with name John
Person node with name Joe
Person node with name Steve
Person node with name Sara
Person node with name Maria


In [9]:
from yfiles_jupyter_graphs import GraphWidget
from google.colab import output
output.enable_custom_widget_manager()
def draw_graph():
    g = session.run("MATCH (n) -[e]-> (m) return n, m, e").graph()
    gw = GraphWidget(graph=g)
    return gw

draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [11]:
results = driver.execute_query("CREATE (j:Person {name: \"Sara\"})-[rel:FRIEND]->(m:Person {name: \"Joe\"})  ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [13]:
results = (driver.execute_query("MATCH (n {name: \"Sara\"}) -[]-> (m {name: \"Joe\"}) RETURN n, m"))
res = results[0][0]
print(res)
print(res["n"].element_id)
print(res["m"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["n"].element_id)
driver.execute_query("MATCH (n) where elementId(n)=$id DETACH DELETE n  ", id=res["m"].element_id)
draw_graph()

<Record n=<Node element_id='4:7ae51117-35f3-4ab8-bab4-353220d429e4:7' labels=frozenset({'Person'}) properties={'name': 'Sara'}> m=<Node element_id='4:7ae51117-35f3-4ab8-bab4-353220d429e4:8' labels=frozenset({'Person'}) properties={'name': 'Joe'}>>
4:7ae51117-35f3-4ab8-bab4-353220d429e4:7
4:7ae51117-35f3-4ab8-bab4-353220d429e4:8


GraphWidget(layout=Layout(height='500px', width='100%'))

In [14]:
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (m:Person {name: \"Joe\"})  MERGE (j)-[r:FRIEND]->(m)  RETURN j, r, m ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [15]:
driver.execute_query("CREATE (aus:Country {name: \"Australia\"})  CREATE (ger:Country {name: \"Germany\"}) ")
driver.execute_query("MATCH (j:Person {name: \"Sara\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Maria\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"John\"})  MATCH (l:Country {name: \"Germany\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Joe\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
driver.execute_query("MATCH (j:Person {name: \"Steve\"})  MATCH (l:Country {name: \"Australia\"})  MERGE (j)-[r:LIVES_IN]->(l)  RETURN j, l ")
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [16]:
results = driver.execute_query("MATCH (p:Person) WHERE p.name =~ 'Jo.*' RETURN p.name  ")
for res in results[0]:
    print(res["p.name"])

John
Joe


In [17]:
results = driver.execute_query("MATCH (p:Person)-[r:FRIEND]->(m:Person)  WHERE exists((m)-[:FRIEND]->(p))  RETURN m, p ")
for res in results[0]:
    print(res["m"]["name"], " -> ", res["p"]["name"])

In [18]:
results = driver.execute_query("MATCH (p1:Person)-[:FRIEND]->(p2:Person) RETURN p1.name, collect(p2.name) as listOfFriends, count(*) as numberOfFriends")
for res in results[0]:
  print("{} has {} friends {}".format(res["p1.name"], res["numberOfFriends"], res["listOfFriends"]))

John has 2 friends ['Joe', 'Sara']
Joe has 1 friends ['Steve']
Sara has 2 friends ['Joe', 'Maria']


**EX1:** Count the number of people who have at least one friend that lives in Australia

**EX2:** Print the names of people who have at least one friend that lives in Australia

In [19]:
q = "TODO - ADD QUERY HERE "
results = driver.execute_query(q)
for res in results[0]:
    print(res)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'TODO': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"TODO - ADD QUERY HERE"
 ^}

In [20]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[relatedTo]-(x)  return x.name, type(relatedTo), Labels(x)[0]")
for res in results[0]:
  print("Maria -[{}]-> {} ({})".format(res["type(relatedTo)"], res["x.name"], res["Labels(x)[0]"]))

Maria -[FRIEND]-> Sara (Person)
Maria -[LIVES_IN]-> Germany (Country)


In [21]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]-(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Sara
Germany
Joe
John


In [22]:
results = driver.execute_query("MATCH (p:Person {name: \"Maria\"})-[*1..2]->(x)  return DISTINCT x ")
for res in results[0]:
  print(res["x"]["name"])

Germany


In [23]:
g = session.run("CALL db.schema.visualization()").graph()
GraphWidget(graph=g)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [24]:
results = driver.execute_query("CALL gds.graph.project(\'projection_name\', [\'Person\', \'Country\'], [\'FRIEND\', \'LIVES_IN\']) YIELD graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipCount AS rels  ")
print(results[0])

[<Record graph='projection_name' nodeProjection={'Country': {'label': 'Country', 'properties': {}}, 'Person': {'label': 'Person', 'properties': {}}} nodes=7 rels=10>]


In [25]:
results = driver.execute_query("CALL gds.pageRank.stream(\'projection_name\') YIELD nodeId, score WITH gds.util.asNode(nodeId) AS n,score AS pageRank RETURN n.name AS name, Labels(n)[0] AS type, pageRank ORDER BY pageRank DESC ")
for res in results[0]:
  print(res)
  print(res["name"], " - ", res["type"], " - ", res["pageRank"])

<Record name='Australia' type='Country' pageRank=0.47173651041666675>
Australia  -  Country  -  0.47173651041666675
<Record name='Germany' type='Country' pageRank=0.4209020833333334>
Germany  -  Country  -  0.4209020833333334
<Record name='Steve' type='Person' pageRank=0.2549927083333334>
Steve  -  Person  -  0.2549927083333334
<Record name='Joe' type='Person' pageRank=0.24704166666666671>
Joe  -  Person  -  0.24704166666666671
<Record name='Maria' type='Person' pageRank=0.2045416666666667>
Maria  -  Person  -  0.2045416666666667
<Record name='Sara' type='Person' pageRank=0.19250000000000003>
Sara  -  Person  -  0.19250000000000003
<Record name='John' type='Person' pageRank=0.15000000000000002>
John  -  Person  -  0.15000000000000002


In [26]:
results = driver.execute_query("CALL gds.pageRank.write('projection_name',     {writeProperty: 'pageRank'} ) YIELD nodePropertiesWritten, ranIterations ")
results = driver.execute_query("MATCH (n) RETURN n.name as name, n.pageRank as pr")
for res in results[0]:
  print(res["name"], " - ", res["pr"])

John  -  0.15000000000000002
Joe  -  0.24704166666666671
Steve  -  0.2549927083333334
Sara  -  0.19250000000000003
Maria  -  0.2045416666666667
Australia  -  0.47173651041666675
Germany  -  0.4209020833333334


In [32]:
results = driver.execute_query("MATCH (source:Person {name: \'John\'}) MATCH (target:Person {name: \'Maria\'}) CALL gds.shortestPath.dijkstra.stream(\'projection_name\', {    sourceNode: source,    targetNode: target}) YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path RETURN    index, gds.util.asNode(sourceNode).name AS SourceName,     gds.util.asNode(targetNode).name AS targetNodeName,    totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,     costs, nodes(path) as path ORDER BY index")
for res in results[0]:
    print("PATH is {} (it costs {})".format(res["nodeNames"], res["costs"]))

PATH is ['John', 'Sara', 'Maria'] (it costs [0.0, 1.0, 2.0])


In [33]:
driver.execute_query("CALL apoc.export.csv.all(null, {stream:TRUE})")

EagerResult(records=[<Record file=None source='database: nodes(7), rels(10)' format='csv' nodes=7 relationships=10 properties=14 time=15 rows=17 batchSize=20000 batches=1 done=True data='"_id","_labels","name","pageRank","_start","_end","_type"\n"0",":Person","John","0.15000000000000002",,,\n"1",":Person","Joe","0.24704166666666671",,,\n"2",":Person","Steve","0.2549927083333334",,,\n"3",":Person","Sara","0.19250000000000003",,,\n"4",":Person","Maria","0.2045416666666667",,,\n"5",":Country","Australia","0.47173651041666675",,,\n"6",":Country","Germany","0.4209020833333334",,,\n,,,,"0","1","FRIEND"\n,,,,"0","3","FRIEND"\n,,,,"0","6","LIVES_IN"\n,,,,"1","2","FRIEND"\n,,,,"1","5","LIVES_IN"\n,,,,"2","5","LIVES_IN"\n,,,,"3","1","FRIEND"\n,,,,"3","4","FRIEND"\n,,,,"3","6","LIVES_IN"\n,,,,"4","6","LIVES_IN"\n'>], summary=<neo4j._work.summary.ResultSummary object at 0x79c9e741b850>, keys=['file', 'source', 'format', 'nodes', 'relationships', 'properties', 'time', 'rows', 'batchSize', 'batches'

In [35]:
driver.execute_query("MATCH (n) DETACH DELETE n ")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x79c9e730b2d0>, keys=[])

In [36]:
driver.execute_query("LOAD CSV WITH HEADERS FROM \"https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/main/data/nodes.csv\" AS line CREATE (:Person {name: line.name, lastName: line.last_name, my_id: line._id}) ")
driver.execute_query("LOAD CSV WITH HEADERS FROM \"https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/main/data/edges.csv\" AS line MATCH (p1:Person {my_id: line.source}) MATCH (p2:Person {my_id: line.target})  MERGE (p1) –[:SUPERVISOR]->(p2) ")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x79c9e749efd0>, keys=[])

In [37]:
draw_graph()

GraphWidget(layout=Layout(height='500px', width='100%'))